In [6]:
!pip3 install -r requirements.txt


You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
path = '/users/sammy/days.txt'
days_file = open(path,'r')
days = days_file.read()

def write_to_file(text, path):
    with open(path, 'w') as file:
        print(appcast, file=appcast_file)
new_days = open(new_path,'w')

title = 'Days of the Week\n'
new_days.write(title)
print(title)

new_days.write(days)
print(days)

days_file.close()
new_days.close()

In [11]:
import csv
import inflect

p = inflect.engine()

def form_singular(word):
    return p.singular_noun(word) or word

occupations = []
salaries = []

with open('datasets/occupation_salaries.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        occupation = row[0] #.strip()
        num_leading_spaces = len(occupation) - len(occupation.lstrip(' '))
        try:
            salary = float(row[1].replace(',',''))
            if (num_leading_spaces == 6) and salary:
                occupation = " ".join(map(form_singular, occupation.split(' ')))
                occupation = occupation.replace('and', 'or')
                occupations.append(occupation)
                salaries.append(salary)
                #print(occupation, salary)
        except ValueError:
            pass
        

normalised_salaries = [(40 * (float(s) / max(salaries))) for s in salaries]
#print('normalised_salaries ', normalised_salaries )
                

In [31]:
from os import path
import pandas as pd
from predict import predict
import plotly.express as px
import plotly.graph_objects as go

def get_most_likely_next_words(previous):
    words = [(word, probability) for word, probability in zip(result['words'], result['probabilities'])]
    sorted_words = sorted(words, key=lambda word: word[1])
    return sorted_words

def get_binary_gender_probabilities(occupation, model_name: str = '345M'):
    ladies_first = predict('A {} is a woman or a man. A {} is a'.format(occupation, occupation), model_name)
    men_just_before = predict('A {} is a man or a woman. A {} is a'.format(occupation, occupation), model_name)
    probability_woman = (ladies_first.get('woman', 0) + men_just_before.get('woman', 0)) / 2
    probability_man = (ladies_first.get('man', 0) + men_just_before.get('man', 0)) / 2
    return {
        'occupation': occupation,
        'genderedness': probability_woman + probability_man,
        'probability_woman': probability_woman,
        'probability_man': probability_man,
        'model_name': model_name
    }

def create_arrow(from_x, from_y, to_x, to_y):
    return go.layout.Annotation(
        x=to_x, y=to_y, xref="x", yref="y", ax=from_x, ay=from_y, axref='x',ayref='y',
        showarrow=True, align="center", arrowhead=2, arrowsize=1, arrowwidth=2, opacity=0.8
    )

def plot_occupations_graph(occupations, salaries=None, file_name=None):
    
    if not salaries:
        salaries = [10 for o in occupations]
        
    occupation_genders_117M = [get_binary_gender_probabilities(occupation, '117M') for occupation in occupations]
    occupation_genders_345M = [get_binary_gender_probabilities(occupation, '345M') for occupation in occupations]
    occupation_genders = occupation_genders_117M + occupation_genders_345M

    plot_occupations = occupations + occupations
    plot_salaries = salaries + salaries

    model_colours = {'117M': 'rgb(215,48,39)', '345M': 'rgb(215,148,39)'}
    colours = [model_colours[g['model_name']] for g in occupation_genders]

    fig = go.Figure()

    fig.update_layout(
        showlegend=False,
        annotations=[
            create_arrow(small['probability_woman'], 
                         small['probability_man'], 
                         medium['probability_woman'], 
                         medium['probability_man']) for small, medium in zip(occupation_genders_117M, occupation_genders_345M)   
        ]
    )

    fig.add_trace(go.Scatter(x=[0,0.3], y=[0,0.3],
                             mode='lines',
                             name='lines'))

    fig.add_trace(go.Scatter(x=[g['probability_woman'] for g in occupation_genders],
                             y=[g['probability_man'] for g in occupation_genders],
                             #mode="markers",
                             mode="markers+text",
                             hovertext=plot_occupations,
                             text=plot_occupations,
                             textfont={
                                #"color": [],
                                #"family": ["Courier New, monospace" for o in plot_occupations],
                                "size": [12 for o in plot_occupations]
                             },
                             textposition=['bottom center' for o in plot_occupations],
                             marker_size=plot_salaries,
                             marker=dict(size=15, color=colours)))
    fig.update_layout(
        #title=go.layout.Title(
        #    text="",
        #    xref="paper",
        #    x=0
        #),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text="Probability Woman",
                font=dict(
                    #family="Courier New, monospace",
                    size=12,
                    color="#7f7f7f"
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text="Probability Man",
                font=dict(
                    #family="Courier New, monospace",
                    size=12,
                    color="#7f7f7f"
                )
            )
        )
    )
    fig.show()

    if file_name:
        with open(path.join('../graphs', file_name), 'w') as file:
            print(fig.to_html(), file=file)

def plot_salaries(occupations, salaries):

    occupation_genders_345M = [get_binary_gender_probabilities(occupation, '345M') for occupation in occupations]
    maleness = (math.atan(1) * 2) / (math.pi)


num_occupations = 50
occupations = occupations[:num_occupations]
salaries = normalised_salaries[:num_occupations]
#plot_occupations_graph(occupations, salaries)

#occupations = ['pilot']
occupations_short = ['pilot', 'surgeon', 'homemaker', 'programmer', 'cleaner', 
                     'president', 'banker', 'footballer', 'politician', 'captain', 'doctor', 'lawyer',
                     'secretary', 'hairdresser', 'model', 'chef', 'cook', 'journalist', 'nurse', 'carer']

plot_occupations_graph(occupations_short, file_name='test.html')
